## Import

In [2]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, Markdown, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 80))

import sys, os, io, time, random, math
import json, base64, requests
import os.path as osp

def _IMPORT_(x):
    try:
        segs = x.split(' ')
        g = globals()
        if 'github.com' in segs[1]:
            uri = segs[1].replace('github.com', 'raw.githubusercontent.com')
            mod = uri.split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif 'gitee.com' in segs[1]:
            mod = segs[1].split('/')
            uri = 'https://' + '/'.join(mod[:-1]) + '/raw/main/' + mod[-1] + '.py'
            x = requests.get(uri).text
        elif segs[1][0] == '/':
            with open(segs[1] + '.py') as fr:
                x = fr.read()
        exec(x, g)
    except:
        pass

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()

def img2b64(x):
    if isinstance(x, bytes):
        return base64.b64encode(x).decode()
    elif isinstance(x, str):
        with open(x, 'rb') as fr:
            return base64.b64encode(fr.read()).decode()
    raise


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed


In [3]:
###
### Common ###
###

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')
_IMPORT_('import ipywidgets as widgets')

# plotly.offline.init_notebook_mode(connected=False)

plt.rcParams['figure.figsize'] = (12.0, 8.0)

def show_table(headers, data, width=900):
    ncols = len(headers)
    width = int(width / ncols)
    lralign = []
    caption = []
    for item in headers:
        astr = ''
        if item[0] == ':':
            astr = ':'
            item = item[1:]
        astr += '---'
        if item[-1] == ':':
            astr += ':'
            item = item[:-1]
        lralign.append(astr)
        caption.append(item)
    captionstr = '|'.join(caption) + chr(10)
    lralignstr = '|'.join(lralign) + chr(10)
    imgholdstr = '|'.join(['<img width=%d/>' % width] * ncols) + chr(10)
    table = captionstr + lralignstr + imgholdstr
    is_dict = isinstance(data[0], dict)
    for row in data:
        if is_dict:
            table += '|'.join([f'{row[c]}' for c in caption]) + chr(10)
        else:
            table += '|'.join([f'{col}' for col in row]) + chr(10)
    return Markdown(table)

def show_video(vidsrc, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if vidsrc.startswith('http'):
        data_url = vidsrc
    else:
        mp4 = open(vidsrc, 'rb').read()
        data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(imgsrc, width=None, height=None):
    if isinstance(imgsrc, np.ndarray):
        img = imgsrc
        if width or height:
            if width and height:
                size = (width, height)
            else:
                rate = img.shape[1] / img.shape[0]
                if width:
                    size = (width, int(width/rate))
                else:
                    size = (int(height*rate), height)
            img = cv2.resize(img, size)
            plt.figure(figsize=(3*int(size[0]/80+1), 3*int(size[1]/80+1)), dpi=80)
        plt.axis('off')
        if len(img.shape) > 2:
            plt.imshow(img);
        else:
            plt.imshow(img, cmap='gray');
        return

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if imgsrc.startswith('http'):
        data_url = imgsrc
    else:
        if len(imgsrc) > 2048:
            data_url = 'data:image/jpg;base64,' + imgsrc
        else:
            img = open(imgsrc, 'rb').read()
            data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    if image_path.startswith('http'):
        data_url = image_path
    else:
        img = open(image_path, 'rb').read()
        data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

def im_read(url, rgb=True, size=None):
    if url.startswith('http'):
        response = requests.get(url)
        if response:
            imgmat = np.frombuffer(response.content, dtype=np.uint8)
            img = cv2.imdecode(imgmat, cv2.IMREAD_COLOR)
        else:
            return None
    else:
        img = cv2.imread(url)
        
    if rgb:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    if size:
        if isinstance(size, int):
            size = (size, size)
        img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    return img


In [5]:
# RACEURL = 'http://116.85.54.39:9119'
RACEURL = 'http://116.85.9.170:9119' # gamma101
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
MODEL_TASK = 'zmq.yolov5.ladder.l.inference'
MSGKEY = 'zmq.yolo.test'
CONF_THRES = 0.25
IOU_THRES = 0.45

## 调试

In [43]:
reqdata = '''{
    "task": "%s",
    "cfg": {
        "pigeon": {
            "msgkey": "%s",
            "user": "1",
            "uuid": "100"
        },
        "data": {
            "class_name": "raceai.data.process.VideoDataLoader",
            "params": {
                "data_source": "live/00cf60a8f512/20210803/videos/20210803104126.mp4",
                "dataset": {
                    "class_name": "raceai.data.VideoFramesDataset",
                    "params": {
                        "data_prefix": "https://frepai.s3.didiyunapi.com",
                        "retrieve_count": 100
                    }
                }
            }
        },
        "nms":{
            "conf_thres": %.2f,
            "iou_thres": %.2f
        }
    }
}''' % (MODEL_TASK, MSGKEY, CONF_THRES, IOU_THRES)

### 输入

只需要修改: `data_source`, `data_prefix`, `retrieve_count`

In [16]:
print(reqdata)

{
    "task": "zmq.yolov5.ladder.l.inference",
    "cfg": {
        "pigeon": {
            "msgkey": "zmq.yolo.test",
            "user": "1",
            "uuid": "100"
        },
        "data": {
            "class_name": "raceai.data.process.VideoDataLoader",
            "params": {
                "data_source": "live/00cf60a8f512/20210803/videos/20210803104126.mp4",
                "dataset": {
                    "class_name": "raceai.data.VideoFramesDataset",
                    "params": {
                        "data_prefix": "https://frepai.s3.didiyunapi.com",
                        "retrieve_count": 50
                    }
                }
            }
        },
        "nms":{
            "conf_thres": 0.25,
            "iou_thres": 0.45
        }
    }
}


In [44]:
reqdata_ = eval(reqdata)
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
json.loads(requests.post(url=API_INFERENCE, json=reqdata_).text)
time.sleep(3)

In [45]:
result = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
result[0]['result'][:10]

[{'image_path': 'frame0.jpg',
  'predict_box': [{'label': 0,
    'conf': '0.821',
    'xyxy': [179, 101, 305, 145]}]},
 {'image_path': 'frame180.jpg',
  'predict_box': [{'label': 0,
    'conf': '0.818',
    'xyxy': [178, 101, 306, 146]}]},
 {'image_path': 'frame360.jpg',
  'predict_box': [{'label': 0,
    'conf': '0.784',
    'xyxy': [176, 102, 305, 143]}]},
 {'image_path': 'frame540.jpg',
  'predict_box': [{'label': 0,
    'conf': '0.812',
    'xyxy': [180, 104, 304, 145]}]},
 {'image_path': 'frame720.jpg',
  'predict_box': [{'label': 0,
    'conf': '0.775',
    'xyxy': [177, 100, 306, 146]}]},
 {'image_path': 'frame900.jpg',
  'predict_box': [{'label': 0,
    'conf': '0.820',
    'xyxy': [179, 101, 306, 145]}]},
 {'image_path': 'frame1080.jpg',
  'predict_box': [{'label': 0,
    'conf': '0.800',
    'xyxy': [177, 100, 307, 146]}]},
 {'image_path': 'frame1260.jpg',
  'predict_box': [{'label': 0,
    'conf': '0.812',
    'xyxy': [179, 101, 304, 144]}]},
 {'image_path': 'frame1440.jpg',

### 输出

通过判断`predict_box`里面有多少个json对象, 每个json对象表示图片中的一个脸, 其中conf表示人脸的置信度, xyxy表示标注人脸的矩形框

```
{
    "pigeon": {
        "msgkey": "zmq.yolov5x.test",
        "user": "1", 
        "uuid": "100"
    }, 
    "task": "zmq.yolov5.ladder.l.inference", 
    "errno": 0, 
    "result": [
        {
            "image_path": "frame_197.jpeg", 
            "predict_box": [
                {
                    "conf": "0.571", 
                    "xyxy": [
                        8, 
                        1, 
                        275, 
                        264
                    ]
                }, 
                {
                    "conf": "0.667", 
                    "xyxy": [
                        268, 
                        90, 
                        403, 
                        325
                    ]
                }, 
                {
                    "conf": "0.859", 
                    "xyxy": [
                        0, 
                        111, 
                        299, 
                        328
                    ]
                }, 
                {
                    "conf": "0.892", 
                    "xyxy": [
                        290, 
                        102, 
                        612, 
                        331
                    ]
                }
            ]
        }, 
        {
            "image_path": "frame_226.jpeg", 
            "predict_box": [
                {
                    "conf": "0.385", 
                    "xyxy": [
                        2228, 
                        624, 
                        3117, 
                        1464
                    ]
                }, 
                {
                    "conf": "0.703", 
                    "xyxy": [
                        1241, 
                        826, 
                        3321, 
                        3000
                    ]
                }, 
                {
                    "conf": "0.854", 
                    "xyxy": [
                        2488, 
                        744, 
                        4137, 
                        3013
                    ]
                }, 
                {
                    "conf": "0.909", 
                    "xyxy": [
                        3769, 
                        739, 
                        5616, 
                        2284
                    ]
                }, 
                {
                    "conf": "0.913", 
                    "xyxy": [
                        216, 
                        1389, 
                        1746, 
                        2969
                    ]
                }, 
                {
                    "conf": "0.928", 
                    "xyxy": [
                        4036, 
                        1365, 
                        5511, 
                        3107
                    ]
                }
            ]
        }
    ]
}
```

## References

- [人脸数据集](https://www.geticsen.cn/Index/article/content/230.html)